In [10]:
import pandas as pd 
import numpy as np 
from matplotlib import pyplot as plt 
import seaborn as sns
import torch 
from sklearn.utils import shuffle
from causalml.inference.meta import BaseXRegressor, BaseRRegressor, BaseSRegressor, BaseTRegressor
from causalml.inference.nn import CEVAE
import dowhy
from dowhy import CausalModel
from sklearn.model_selection import train_test_split

### Load datasets 

In [115]:
# load all lalonde data

nsw_synthetic_controls = pd.read_csv("/Users/mawuliagamah/gitprojects/causal_inference/causal_inference/datasets/lalonde_synthetic_control.csv")

nsw_synthetic_controls.drop('propensity_score', axis=1, inplace=True)
nsw_synthetic_controls.drop('propensity_logit', axis=1, inplace=True)
nsw_synthetic_controls.drop('Unnamed: 0', axis=1, inplace=True)


nswre74_control = pd.read_excel('/Users/mawuliagamah/gitprojects/causal_inference/causal_inference/datasets/nswre74_control.xls')
nswre74_treated = pd.read_excel('/Users/mawuliagamah/gitprojects/causal_inference/causal_inference/datasets/nswre74_treated.xlsx')

nswre74_pd_df = pd.concat([nswre74_control,nswre74_treated]) #Combine treatment into a single control as pandas dataframe

#Shuffle data sets 
nswre74_pd_df = shuffle(nswre74_pd_df).reset_index(drop=True)
nsw_synthetic_controls

,treat,age,education,black,hispanic,married,nodegree,re74,re75,re78
0,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.000000,0.000000,9930.046000
1,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.000000,0.000000,3595.894000
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.000000,0.000000,24909.450000
3,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.000000,0.000000,7506.146000
4,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.000000,0.000000,289.789900
...,...,...,...,...,...,...,...,...,...,...
3615,0.0,40.0,16.0,0.0,0.0,1.0,0.0,25862.320312,25243.550781,25564.669922
3616,0.0,40.0,16.0,0.0,0.0,1.0,0.0,25862.320312,25243.550781,25564.669922
3617,0.0,40.0,16.0,0.0,0.0,1.0,0.0,25862.320312,25243.550781,25564.669922
3618,0.0,27.0,12.0,0.0,0.0,1.0,0.0,14812.059570,17670.480469,25564.669922


In [116]:
# Define binary features
binfeatures = ['black','hispanic','married','nodegree']
confeatures = ['age','education','re74','re75']
perm = binfeatures + confeatures
# Variables 
common_causes = "age, education, black, hispanic, married, nodegree, re75,re78"


In [117]:
df = nsw_synthetic_controls.reset_index(drop=True)
df.head()

,treat,age,education,black,hispanic,married,nodegree,re74,re75,re78
0,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.0,0.0,9930.0460
1,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.0,0.0,3595.8940
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,0.0,24909.4500
3,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.0,0.0,7506.1460
4,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.0,0.0,289.7899


In [118]:
FEATURES = df[perm].values
TREATMENT = df['treat'].values
OUTCOME = df['re78']


In [119]:
# Train and evaluation index
trainidx , evalidx  = train_test_split(np.arange(FEATURES.shape[0]), test_size=0.3, random_state=42)
# Sample data 
FEATURES_train, OUTCOME_train,TREATMENT_train = FEATURES[trainidx], OUTCOME[trainidx], TREATMENT[trainidx]
FEATURES_valid, OUTCOME_valid ,TREATMENT_valid, = FEATURES[evalidx],OUTCOME[evalidx], TREATMENT[evalidx]

## CEVAE model

In [147]:
outcome_dist = "normal"
latent_dim = 20
hidden_dim = 15
num_epochs = 5
batch_size = 10
learning_rate = 0.0001
learning_rate_decay = 0.1
num_layers = 2
num_samples = 1

In [148]:
# create tensors 
FEATURES = torch.tensor(FEATURES_train, dtype=torch.float)
TREATMENT=torch.tensor(TREATMENT_train, dtype=torch.float)
OUTCOME = torch.tensor(OUTCOME_train.values, dtype=torch.float)

In [149]:
cevae = CEVAE(outcome_dist=outcome_dist,
              latent_dim=latent_dim,
              hidden_dim=hidden_dim,
              num_epochs=num_epochs,
              batch_size=batch_size,
              learning_rate=learning_rate,
              learning_rate_decay=learning_rate_decay,
              num_layers=num_layers,
              num_samples=num_samples)

In [150]:
# fit model 
losses = cevae.fit(FEATURES,TREATMENT,OUTCOME )

To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
INFO 	 Training with 254 minibatches per epoch
DEBUG 	 step     0 loss = 1.23619e+10
DEBUG 	 step   100 loss = 6.74736e+12
DEBUG 	 step   200 loss = 7.93913e+10
DEBUG 	 step   300 loss = 5.24371e+13
DEBUG 	 step   400 loss = 1.93737e+13
DEBUG 	 step   500 loss = 3.27431e+13


KeyboardInterrupt: 

In [151]:
ite = cevae.(FEATURES_valid)
ate = ite.mean() 

AttributeError: 'CEVAE' object has no attribute 'ite'

In [ ]:
CEVAE.fi